# Export analysis grid tiles from an `odc-geo` GridSpec

## Getting started
Set working directory to top level of repo to ensure links work correctly:

In [ ]:
cd ../..

Install additional packages directly from the requirements file

In [ ]:
pip install -r requirements.in --quiet

In [ ]:
import pandas as pd
import geopandas as gpd

from odc.geo.geom import BoundingBox
from odc.geo.gridspec import GridSpec
from odc.geo.types import xy_

from intertidal.io import _id_to_tuple


## Create GridSpec tile grid

In [ ]:
# Load Collection 3 summary grid, reproject to Albers
c3_path = "https://data.dea.ga.gov.au/derivative/ga_summary_grid_c3.geojson"
c3_grid = gpd.read_file(c3_path)

# Extract BoundingBox, converting to ints to avoid floating point issues
c3_bbox = BoundingBox(
    *c3_grid.to_crs("EPSG:3577").total_bounds.round().astype(int), crs="EPSG:3577"
)
c3_bbox

In [ ]:
# Create 32 km tiles using gridspec, using origin of updated AOI grid
gs_32km = GridSpec(
    crs="EPSG:3577",
    resolution=10,
    tile_shape=(3200, 3200),
    origin=xy_(-4416000, -6912000),  # near Heard and McDonald Islands
)
gs_32km[0, 0]

## Create vector file outputs

In [ ]:
# Convert grid to geopandas polygon dataset covering full C3 grid extent
grid_gdf = gpd.GeoDataFrame.from_features(
    gs_32km.geojson(bbox=c3_bbox), crs="EPSG:4326"
).to_crs("EPSG:3577")

# Update column names to match current C3 grid formatting
grid_gdf[["ix", "iy"]] = grid_gdf.idx.str.split(",", expand=True).astype(int)
grid_gdf["region_code"] = (
    "x"
    + grid_gdf.ix.astype(str).str.zfill(3)
    + "y"
    + grid_gdf.iy.astype(str).str.zfill(3)
)

# Return only selected columns
grid_gdf = grid_gdf[["region_code", "ix", "iy", "geometry"]]

In [ ]:
# grid_gdf.explore()

### Join GEODATA Topo 100k attributes

In [ ]:
region_path = "/gdata1/data/boundaries/GEODATA_COAST_100K/australia/cstauscd_r.shp"
region_gdf = gpd.read_file(region_path).to_crs("EPSG:3577")
region_gdf = region_gdf[["FEAT_CODE", "geometry"]].set_index("FEAT_CODE")


In [ ]:
# Identify mainland tiles, and add to "type" field
grid_gdf_mainland = grid_gdf.clip(region_gdf.loc[["mainland"]])
grid_gdf["type"] = "offshore"
grid_gdf.loc[grid_gdf_mainland.index, "type"] = "mainland"

### Subset and clip to the coast

#### Load individual datasets

In [ ]:
# GA GEODATA COAST 100K "coastline" features
goadata100k_gdf = (
    gpd.read_file("/gdata1/data/boundaries/GEODATA_COAST_100K/australia/cstauscd_l.shp")
    .query("FEAT_CODE == 'coastline'")
    .to_crs("EPSG:3577")
)

In [ ]:
# Non-"mainland" features from Complete Great Barrier Reef (GBR) Island
# and Reef Feature boundaries including Torres Strait Version 1b
# https://eatlas.org.au/data/uuid/d2396b2c-68d4-4f4b-aab0-52f7bc4a81f5
gbrreefs_gdf = (
    gpd.read_file(
        "/gdata1/data/reefs/TS_AIMS_NESP_Torres_Strait_Features_V1b_with_GBR_Features.shp"
    )
    .query("FEAT_NAME != 'Mainland'")
    .to_crs("EPSG:3577")
)
gbrreefs_gdf["geometry"] = gbrreefs_gdf.boundary

In [ ]:
# Coral Sea reefs (-10 m contour) from Coral Sea features satellite 
# imagery and raw depth contours (Sentinel 2 and Landsat 8) 2015 – 2021 
# https://eatlas.org.au/data/uuid/df5a5b47-ad4c-431e-be49-af52f64aafce
coralseareefs_gdf = (
    gpd.read_file("/gdata1/data/reefs/CS_AIMS_Coral-Sea-Features_Img_L8_R1_Depth10m_Coral-Sea.shp")
    .to_crs("EPSG:3577")
)

In [ ]:
# Additional manually drawn features to capture remote reefs off WA/Qld
additionalislands_gdf = (
    gpd.read_file(
        "data/raw/additional_islands.geojson"
    )
    .to_crs("EPSG:3577"))


#### Combine and select coastal grid cells

In [ ]:
# Combine into a single dataset
coastlines_gdf = pd.concat(
    [
        goadata100k_gdf[["geometry"]],
        gbrreefs_gdf[["geometry"]],
        coralseareefs_gdf[["geometry"]],
        additionalislands_gdf[["geometry"]],
    ],
)

# Keep only grid cells within 1000 m of coastlines
grid_gdf_coastal = (
    gpd.sjoin_nearest(grid_gdf, coastlines_gdf, max_distance=1000)
    .groupby("region_code")
    .first()
    .drop("index_right", axis=1)
    .set_crs("EPSG:3577")
)
grid_gdf_coastal

In [ ]:
grid_gdf_coastal.plot()

In [ ]:
coastlines_gdf.to_crs("EPSG:3577").to_file("test.geojson")

#### Clip grid cells (to do)

In [ ]:
# mask_path = "https://data.dea.ga.gov.au/derivative/dea_coastlines/supplementary/albers_grids/coastal_mask_15km.geojson"
# coastal_mask_gdf = gpd.read_file(mask_path)

In [ ]:
# # Clip to coastal zone using mask
# grid_gdf_clipped = grid_gdf_joined.clip(coastal_mask_gdf)

### Export all outputs

In [ ]:
grid_gdf.to_crs("EPSG:4326").to_file("data/raw/ga_summary_grid_c3_32km.geojson")
grid_gdf_coastal.to_crs("EPSG:4326").to_file("data/raw/ga_summary_grid_c3_32km_coastal.geojson")
# grid_gdf_clipped.to_file("ga_summary_grid_c3_32km_coastal_clipped.geojson")

## Test GridSpec

In [ ]:
# Test the gridspec on a tile ID
gs_32km[id_to_tuple("x135y101")]

In [ ]:
# Test gridspec on coordinates
gs_32km[gs_32km.pt2idx(x=1904005, y=-3664005)]